In [43]:
# !pip uninstall gensim -y 
!pip install gensim #==3.8.3 #4.3.3
!pip install scikit-learn
!pip install numpy
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121
# !pip uninstall tensorflow -y
!pip install tensorflow==2.17.0
!pip install matplotlib

Looking in indexes: https://download.pytorch.org/whl/cu121


In [12]:
import numpy as np

from modules.gnn_models import MLPPredictor

import torch

In [44]:
TOR_ORIG_LABELS_DICT = {'P2P':0, 'C2P': 1,'Siblings': 2, 'P2C': 3}
TOR_CSV_LABELS_DICT = {'P2P':0,'P2C': -1}
DATA_PATH = 'data/CAIDA_AS_Relationships/Serial_1/'
PATH = "/home/valentina/Desktop/GIT/BenchmarckASRelationships/"
class_names = ['P2P', 'C2P', 'P2C']
DATA = 'data/'

# Importar ToR Dataset

In [45]:
# Cargar x_training y x_test
x_training = np.load(PATH + DATA + "x_training.npy").astype(int)
x_test = np.load(PATH + DATA + "x_test.npy").astype(int)


# Cargar y_training y y_test
y_training = np.load(PATH + DATA + "y_training.npy").astype(int)
y_test = np.load(PATH + DATA + "y_test.npy").astype(int)


print("x_training.shape:", x_training.shape)
print("y_training.shape:", y_training.shape)
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)

print('x_training:',x_training)
print('y_training:',y_training)
print('x_test:',x_test)
print('y_test:',y_test)

x_training.shape: (795638, 2)
y_training.shape: (795638,)
x_test.shape: (198910, 2)
y_test.shape: (198910,)
x_training: [[ 59675  18106]
 [ 11232   6939]
 [201806 199524]
 ...
 [ 38522 209102]
 [270362 271253]
 [ 15305 395846]]
y_training: [0 1 0 ... 0 0 2]
x_test: [[263152  53180]
 [  3214  45758]
 [ 34019   8529]
 ...
 [ 37658  16552]
 [  3356   9930]
 [138064  59605]]
y_test: [0 0 0 ... 0 0 0]


# Importar Embeddings

In [65]:
import torch

# Cargar el tensor de embeddings
embeddings_pt = torch.load('embeddings.pt')  # asegúrate que sea un tensor
total_ASNs, embedding_vecor_length = embeddings_pt.shape
embeddings_np = embeddings_pt.detach().numpy()

print(f"[TOTAL ASN]: {total_ASNs}" )
print(f"[EMBEDDINGS LEN]: {embedding_vecor_length}")
print(f"[EMBEDDINGS] {embeddings_pt}")


[TOTAL ASN]: 74140
[EMBEDDINGS LEN]: 16
[EMBEDDINGS] tensor([[ 2.9606, -0.5309, -0.0631,  ...,  5.7811,  2.5728,  0.4085],
        [ 3.4744, -0.5007,  0.5984,  ...,  5.2775,  2.6044, -0.0717],
        [ 3.6919, -0.6067, -1.1352,  ...,  6.2019,  2.6469,  0.9469],
        ...,
        [ 4.6999,  1.7849, -6.6735,  ...,  6.5724,  1.7841,  2.3854],
        [ 5.0911,  0.4358, -3.0184,  ...,  5.7703,  1.8840, -0.7117],
        [-0.5820, -0.7045, -0.8149,  ...,  2.3244, -0.5679,  1.2785]],
       requires_grad=True)


los modelos de embeddings trabajan con índices, no con ASNs reales.

En este caso como creamos los embeddings a partir del grafo DGL en el archivo train_embeddings, sabemos que se guardan todos los embeddings desde 0 hasta el mayor ASN, por lo talto sabemos que la matriz guardada en la posicion de las filas coincidira con su ASN. (porque asociamos el numero de nodo con su ASN)

In [ ]:
asn_to_index = {}
for  i, asn in enumerate(embeddings_np):
    asn_to_index[i] = i
# FIXME: Esto no tiene sentido ... pero necesito esto para fiotrar mas

# print(f"[ASN TO INDEX]: {asn_to_index}")


Ahora en ves de asocian que ASN_a con ASN_b estan conectados tendromos
que el indice_ASN_b esta conectado con indice_ASN_b?c en los datos de training y test de ToR

In [71]:
# ASN a índice -> x_training y x_test
x_training_mapped = []
y_training_filtered = []

# Recorrer par (asn, asn) y agregamos válidos
for (a, b), y in zip(x_training, y_training):
    a_index = asn_to_index.get(str(a), 0)
    b_index = asn_to_index.get(str(b), 0)

    # Ignorar si índice es 0 
    if a_index == 0 or b_index == 0 :
        continue
    
    x_training_mapped.append([a_index, b_index])
    y_training_filtered.append(y)

# Convertir listas a arrays de NumPy
x_training_mapped = np.array(x_training_mapped)
y_training_filtered = np.array(y_training_filtered)

# ---------------------------------------------------------------------------

x_test_mapped = []
y_test_filtered = []

for (a, b), y in zip(x_test, y_test):
    a_index = asn_to_index.get(str(a), 0)
    b_index = asn_to_index.get(str(b), 0)

    # Ignorar si índice es 0 
    if a_index == 0 or b_index == 0:
        continue
    
    x_test_mapped.append([a_index, b_index])
    y_test_filtered.append(y)

x_test_mapped = np.array(x_test_mapped)
y_test_filtered = np.array(y_test_filtered)

# Guardar los datos corregidos
np.save(DATA + "x_training_mapped.npy", x_training_mapped)
np.save(DATA + "y_training_filtered.npy", y_training_filtered)
np.save(DATA + "x_test_mapped.npy", x_test_mapped)
np.save(DATA + "y_test_filtered.npy", y_test_filtered)


# Modelo NN de BGP2Vec

In [72]:
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten
# # from keras.layers import LSTM
from keras.layers import Embedding
# # from keras.preprocessing import sequence
# # from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [62]:
print("[SHAPE EMDEDINGS]",embeddings_pt.shape)

[SHAPE EMDEDINGS] torch.Size([74140, 16])


In [73]:
# Configuración de la RNN BGP2VEC
experiment = None
num_classes = len(class_names)
embedding_trainable = False
input_length = 2  # longitud de la secuencia de entrada
embedding_vector_length = embeddings_pt.shape[1]  # tamaño de los vectores de embedding
total_ASNs = embeddings_pt.shape[0] #42865 # Tamaño del vocabulario


# Registro de parámetros del experimento si es aplicable
if experiment is not None:
    experiment.log_parameter("embedding_trainable", embedding_trainable)

# Definición del modelo
model = Sequential()
model.add(Embedding(total_ASNs, embedding_vector_length, input_length=input_length,
                    weights=[embeddings_np], trainable=embedding_trainable))  # Asegúrate de que 'embeddings' esté definida
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))

model.add(Reshape((32, 2)))  # Cambia Reshape para reflejar las dimensiones correctas
#model.add(Reshape((model.output_shape[2], model.output_shape[1])))  # Cambia Reshape para reflejar las dimensiones correctas
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Reshape((32,16)))  # Cambia Reshape para reflejar las dimensiones correctas
#model.add(Reshape((model.output_shape[2], model.output_shape[1])))  # Cambia Reshape para reflejar las dimensiones correctas
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

/home/valentina/Desktop/GIT/TrabajoTesis/env310/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [74]:
# Compilación del modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Resumen del modelo
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │     1,186,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,186,240 (4.53 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,186,240 (4.53 MB)

None


In [75]:

# Convertir las clases a un one-hot vector
y_training_vector = to_categorical(y_training_filtered, num_classes) 
y_test_vector = to_categorical(y_test_filtered, num_classes)
print("[Y TRAINING VECTOR]",y_training_vector)
print("[X TRAINING]",x_training)

[Y TRAINING VECTOR] []
[X TRAINING] [[ 59675  18106]
 [ 11232   6939]
 [201806 199524]
 ...
 [ 38522 209102]
 [270362 271253]
 [ 15305 395846]]


In [ ]:
class_names = ['P2P', 'C2P', 'P2C']

In [76]:
# Use class_weight to deal with unbalanced dataset
from sklearn.utils import class_weight

# class_weights = class_weight.compute_class_weight('balanced', list(range(num_classes)), y_training)
# print(class_weights)
class_weights = None

In [78]:
# Fit model
import os
MODEL_NAME = "NN_GNN"
checkpointer_acc = ModelCheckpoint(monitor='val_acc', filepath= os.getcwd() + MODEL_NAME + '_acc.weights.h5',
                                    verbose=1, 
                                    save_best_only=True, 
                                    save_weights_only=True)

checkpointer_loss = ModelCheckpoint(filepath= os.getcwd() + MODEL_NAME + '_loss.weights.h5', 
                                    verbose=1, 
                                    save_best_only=True, 
                                    save_weights_only=True)

callbacks = [checkpointer_loss,checkpointer_acc] #tensorboard


In [ ]:
import math
experiment = None
epochs = 1
batch_size = 64
steps_per_epoch = math.ceil(len(x_training)/batch_size)
val_batch_size = 64
validation_steps = math.ceil(len(x_test)/val_batch_size)



def generator(features, labels, batch_size):
    while True:
        for i in range(0, len(features), batch_size):
            batch_features = features[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]

            # Evitar que devuelva batches incompletos
            if len(batch_features) != batch_size:
                continue  
            
            yield np.array(batch_features), np.array(batch_labels)


def val_generator(features, labels, batch_size):
    while True:
        for i in range(0, len(features), batch_size):
            batch_features = features[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]

            if len(batch_features) != batch_size:
                continue  
            
            yield np.array(batch_features), np.array(batch_labels)

In [82]:
history = model.fit(
        x=val_generator(x_training_mapped, y_training_vector, batch_size),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weights,
        validation_data=val_generator(x_test_mapped, y_test_vector, val_batch_size),
        validation_steps=validation_steps
    )

KeyboardInterrupt: 

In [ ]:
history = model.fit(
    x=val_generator(x_training_mapped, y_training_vector, batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=callbacks,
    class_weight=class_weights,
    validation_data=val_generator(x_test_mapped, y_test_vector, val_batch_size),
    validation_steps=validation_steps,
    verbose=1  # <-- Agrega esto
)


In [84]:
from IPython.display import display


In [ ]:
test_scores = model.evaluate(x_test_mapped, y_test_vector, batch_size=val_batch_size, verbose=1)
print("Accuracy: %.2f%%" % (test_scores[1]*100))

In [ ]:
# y_test_prediction = model.predict_classes(x_test, batch_size=val_batch_size, verbose=1)
y_test_predictions_prob = model.predict(x_test_mapped, batch_size=val_batch_size, verbose=1)
# Obtener pradicciones de las clases
y_test_prediction = np.argmax(y_test_predictions_prob, axis=1)

In [ ]:
# y_training_prediction = model.predict_classes(x_training, batch_size=val_batch_size, verbose=1)
y_training_predictions_prob = model.predict(x_training_mapped, batch_size=val_batch_size, verbose=1)
# Obtener las clases predichas
y_training_prediction = np.argmax(y_training_predictions_prob, axis=1)

In [ ]:
# Guardamos los resultados en bgp2vec_results_tor.txt

with open("bgp2vec_results_tor.txt", "w") as f:
    # AS1 | AS2 | Predicción | Real
    for i in range(len(y_test_prediction)):
        f.write(f"{x_test_mapped[i][0]}|{x_test_mapped[i][1]}|{y_test_prediction[i]}|{y_test[i]} \n")

print("Archivo bgp2vec_results_tor.txt guardado correctamente.")

## Plot History Accuracy

In [ ]:
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth


with open(os.getcwd() + "/results/results_" + MODEL_NAME +  "_accuracy.pkl", 'wb') as output:
    pickle.dump(history.history, output, pickle.HIGHEST_PROTOCOL)
    if experiment is not None:
        experiment.log_asset(os.getcwd() + "/results/results_" + MODEL_NAME +  "_accuracy.pkl")
    
# list all data in history
print(history.history.keys())
x = np.asarray(range(1,epochs + 1))
# summarize history for accuracy
plt.figure()
plt.plot(x, smooth([y*100 for y in history.history['accuracy']],2))
# plt.plot(x, [y*100 for y in history_history['val_acc']])
plt.plot(x, smooth([y*100 for y in history.history['val_accuracy']],2))
plt.ylabel('Accuracy (%)')
plt.xlabel('Epochs')
plt.ylim(50,100) ###########################
plt.legend(['Training', 'Test'], loc='lower right')
plt.grid()
plt.show()


## Plot and save a confusion matrix for results over the test set

In [ ]:
%matplotlib inline

import matplotlib
import pylab as pl
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          fname='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
        plt.imshow([[100*j for j in i] for i in cm], interpolation='nearest', cmap=cmap)
        cbar = plt.colorbar()
        cbar.ax.set_yticklabels(['0%','20%','40%','60%','80%','100%'])
    else:
        print('Confusion matrix, without normalization')
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, format(cm[i, j]*100, fmt) + '%',
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")    
        
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test_filtered, y_test_prediction)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized Confusion Msatrix')


plt.show()